# Úloha klasifikácie textu

Ako sme už spomenuli, zameriame sa na jednoduchú úlohu klasifikácie textu založenú na dátovom súbore **AG_NEWS**, kde je cieľom klasifikovať nadpisy správ do jednej zo 4 kategórií: Svet, Šport, Biznis a Veda/Technológie.

## Dátový súbor

Tento dátový súbor je súčasťou modulu [`torchtext`](https://github.com/pytorch/text), takže k nemu máme jednoduchý prístup.


In [1]:
import torch
import torchtext
import os
import collections
os.makedirs('./data',exist_ok=True)
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
classes = ['World', 'Sports', 'Business', 'Sci/Tech']

Tu `train_dataset` a `test_dataset` obsahujú kolekcie, ktoré vracajú dvojice štítku (číslo triedy) a textu, napríklad:


In [2]:
list(train_dataset)[0]

(3,
 "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.")

Takže, poďme vytlačiť prvých 10 nových titulkov z nášho datasetu:


In [5]:
for i,x in zip(range(5),train_dataset):
    print(f"**{classes[x[0]]}** -> {x[1]}")


**Sci/Tech** -> Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again.
**Sci/Tech** -> Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\which has a reputation for making well-timed and occasionally\controversial plays in the defense industry, has quietly placed\its bets on another part of the market.
**Sci/Tech** -> Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\about the economy and the outlook for earnings are expected to\hang over the stock market next week during the depth of the\summer doldrums.
**Sci/Tech** -> Iraq Halts Oil Exports from Main Southern Pipeline (Reuters) Reuters - Authorities have halted oil export\flows from the main pipeline in southern Iraq after\intelligence showed a rebel militia could strike\infrastructure, an oil official said on Saturday.
**Sci/Tech** -> Oil prices soar to

Pretože datasety sú iterátory, ak chceme použiť údaje viackrát, musíme ich previesť na zoznam:


In [3]:
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
train_dataset = list(train_dataset)
test_dataset = list(test_dataset)

## Tokenizácia

Teraz musíme previesť text na **čísla**, ktoré môžu byť reprezentované ako tenzory. Ak chceme reprezentáciu na úrovni slov, musíme urobiť dve veci:
* použiť **tokenizér** na rozdelenie textu na **tokeny**
* vytvoriť **slovník** týchto tokenov.


In [4]:
tokenizer = torchtext.data.utils.get_tokenizer('basic_english')
tokenizer('He said: hello')

['he', 'said', 'hello']

In [5]:
counter = collections.Counter()
for (label, line) in train_dataset:
    counter.update(tokenizer(line))
vocab = torchtext.vocab.vocab(counter, min_freq=1)

Pomocou slovnej zásoby môžeme ľahko zakódovať tokenizovaný reťazec do množiny čísel:


In [19]:
vocab_size = len(vocab)
print(f"Vocab size if {vocab_size}")

stoi = vocab.get_stoi() # dict to convert tokens to indices

def encode(x):
    return [stoi[s] for s in tokenizer(x)]

encode('I love to play with my words')

Vocab size if 95810


[599, 3279, 97, 1220, 329, 225, 7368]

## Reprezentácia textu pomocou Bag of Words

Keďže slová nesú význam, niekedy dokážeme pochopiť význam textu len na základe jednotlivých slov, bez ohľadu na ich poradie vo vete. Napríklad pri klasifikácii správ môžu slová ako *počasie*, *sneh* naznačovať *predpoveď počasia*, zatiaľ čo slová ako *akcie*, *dolár* by mohli odkazovať na *finančné správy*.

**Bag of Words** (BoW) je najčastejšie používaná tradičná vektorová reprezentácia. Každé slovo je priradené k indexu vektora a prvok vektora obsahuje počet výskytov daného slova v konkrétnom dokumente.

![Obrázok znázorňujúci, ako je reprezentácia Bag of Words uložená v pamäti.](../../../../../translated_images/sk/bag-of-words-example.606fc1738f1d7ba9.webp) 

> **Note**: Na BoW sa môžete pozerať aj ako na súčet všetkých one-hot-enkódovaných vektorov pre jednotlivé slová v texte.

Nižšie je uvedený príklad, ako vytvoriť reprezentáciu Bag of Words pomocou knižnice Scikit Learn v jazyku Python:


In [7]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[1, 1, 0, 2, 0, 0, 0, 0, 0]], dtype=int64)

Na výpočet vektora bag-of-words z vektorovej reprezentácie našej AG_NEWS dátovej sady môžeme použiť nasledujúcu funkciu:


In [20]:
vocab_size = len(vocab)

def to_bow(text,bow_vocab_size=vocab_size):
    res = torch.zeros(bow_vocab_size,dtype=torch.float32)
    for i in encode(text):
        if i<bow_vocab_size:
            res[i] += 1
    return res

print(to_bow(train_dataset[0][1]))

tensor([2., 1., 2.,  ..., 0., 0., 0.])


> **Poznámka:** Tu používame globálnu premennú `vocab_size` na určenie predvolenej veľkosti slovníka. Keďže veľkosť slovníka je často dosť veľká, môžeme obmedziť veľkosť slovníka na najčastejšie slová. Skúste znížiť hodnotu `vocab_size` a spustiť kód nižšie, aby ste videli, ako to ovplyvní presnosť. Mali by ste očakávať určitý pokles presnosti, ale nie dramatický, na úkor vyššieho výkonu.


## Trénovanie klasifikátora BoW

Teraz, keď sme sa naučili, ako vytvoriť reprezentáciu Bag-of-Words pre náš text, poďme na nej natrénovať klasifikátor. Najprv musíme upraviť náš dataset na trénovanie tak, aby všetky pozičné vektorové reprezentácie boli konvertované na reprezentáciu bag-of-words. To môžeme dosiahnuť tým, že funkciu `bowify` použijeme ako parameter `collate_fn` v štandardnom torch `DataLoader`:


In [21]:
from torch.utils.data import DataLoader
import numpy as np 

# this collate function gets list of batch_size tuples, and needs to 
# return a pair of label-feature tensors for the whole minibatch
def bowify(b):
    return (
            torch.LongTensor([t[0]-1 for t in b]),
            torch.stack([to_bow(t[1]) for t in b])
    )

train_loader = DataLoader(train_dataset, batch_size=16, collate_fn=bowify, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, collate_fn=bowify, shuffle=True)

Teraz definujme jednoduchú klasifikačnú neurónovú sieť, ktorá obsahuje jednu lineárnu vrstvu. Veľkosť vstupného vektora sa rovná `vocab_size` a veľkosť výstupu zodpovedá počtu tried (4). Keďže riešime klasifikačnú úlohu, konečná aktivačná funkcia je `LogSoftmax()`.


In [22]:
net = torch.nn.Sequential(torch.nn.Linear(vocab_size,4),torch.nn.LogSoftmax(dim=1))

Teraz definujeme štandardný tréningový cyklus v PyTorch. Keďže náš dataset je pomerne veľký, na účely výučby budeme trénovať iba jednu epochu, a niekedy dokonca menej ako jednu epochu (určenie parametra `epoch_size` nám umožňuje obmedziť tréning). Počas tréningu budeme tiež hlásiť akumulovanú tréningovú presnosť; frekvencia hlásenia je určená pomocou parametra `report_freq`.


In [24]:
def train_epoch(net,dataloader,lr=0.01,optimizer=None,loss_fn = torch.nn.NLLLoss(),epoch_size=None, report_freq=200):
    optimizer = optimizer or torch.optim.Adam(net.parameters(),lr=lr)
    net.train()
    total_loss,acc,count,i = 0,0,0,0
    for labels,features in dataloader:
        optimizer.zero_grad()
        out = net(features)
        loss = loss_fn(out,labels) #cross_entropy(out,labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss
        _,predicted = torch.max(out,1)
        acc+=(predicted==labels).sum()
        count+=len(labels)
        i+=1
        if i%report_freq==0:
            print(f"{count}: acc={acc.item()/count}")
        if epoch_size and count>epoch_size:
            break
    return total_loss.item()/count, acc.item()/count

In [25]:
train_epoch(net,train_loader,epoch_size=15000)

3200: acc=0.8028125
6400: acc=0.8371875
9600: acc=0.8534375
12800: acc=0.85765625


(0.026090790722161722, 0.8620069296375267)

## BiGramy, TriGramy a N-Gramy

Jedným z obmedzení prístupu "bag of words" je, že niektoré slová sú súčasťou viacslovných výrazov. Napríklad slovo 'hot dog' má úplne iný význam ako slová 'hot' a 'dog' v iných kontextoch. Ak by sme slová 'hot' a 'dog' vždy reprezentovali rovnakými vektormi, mohlo by to zmiasť náš model.

Na riešenie tohto problému sa často používajú **N-gramové reprezentácie** v metódach klasifikácie dokumentov, kde frekvencia každého slova, dvojslovného alebo trojslovného výrazu predstavuje užitočný prvok pre trénovanie klasifikátorov. Napríklad v bigramovej reprezentácii pridáme do slovníka všetky dvojice slov, okrem pôvodných slov.

Nižšie je uvedený príklad, ako vytvoriť bigramovú reprezentáciu "bag of words" pomocou Scikit Learn:


In [26]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=1)
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
bigram_vectorizer.fit_transform(corpus)
print("Vocabulary:\n",bigram_vectorizer.vocabulary_)
bigram_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()


Vocabulary:
 {'i': 7, 'like': 11, 'hot': 4, 'dogs': 2, 'i like': 8, 'like hot': 12, 'hot dogs': 5, 'the': 16, 'dog': 0, 'ran': 14, 'fast': 3, 'the dog': 17, 'dog ran': 1, 'ran fast': 15, 'its': 9, 'outside': 13, 'its hot': 10, 'hot outside': 6}


array([[1, 0, 1, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int64)

Hlavnou nevýhodou prístupu N-gram je, že veľkosť slovníka začína rásť extrémne rýchlo. V praxi je potrebné kombinovať reprezentáciu N-gram s niektorými technikami redukcie dimenzií, ako sú *embeddingy*, o ktorých budeme diskutovať v nasledujúcej jednotke.

Aby sme mohli použiť reprezentáciu N-gram v našej **AG News** dátovej sade, musíme vytvoriť špeciálny ngram slovník:


In [27]:
counter = collections.Counter()
for (label, line) in train_dataset:
    l = tokenizer(line)
    counter.update(torchtext.data.utils.ngrams_iterator(l,ngrams=2))
    
bi_vocab = torchtext.vocab.vocab(counter, min_freq=1)

print("Bigram vocabulary length = ",len(bi_vocab))

Bigram vocabulary length =  1308842


Mohli by sme použiť ten istý kód ako vyššie na trénovanie klasifikátora, avšak bolo by to veľmi neefektívne z hľadiska pamäte. V ďalšej jednotke budeme trénovať bigramový klasifikátor pomocou embeddingov.

> **Poznámka:** Môžete ponechať iba tie ngramy, ktoré sa v texte vyskytujú viac ako určený počet krát. Tým sa zabezpečí, že zriedkavé bigramy budú vynechané, a výrazne sa zníži dimenzionalita. Na tento účel nastavte parameter `min_freq` na vyššiu hodnotu a sledujte zmenu dĺžky slovníka.


## Term Frequency Inverse Document Frequency TF-IDF

V reprezentácii BoW sú výskyty slov rovnomerne vážené, bez ohľadu na samotné slovo. Je však zrejmé, že časté slová, ako napríklad *a*, *v*, atď., sú oveľa menej dôležité pre klasifikáciu než špecializované výrazy. V skutočnosti sú v mnohých úlohách NLP niektoré slová relevantnejšie než iné.

**TF-IDF** znamená **frekvencia termínu–inverzná frekvencia dokumentu**. Ide o variáciu metódy bag of words, kde namiesto binárnej hodnoty 0/1, ktorá označuje prítomnosť slova v dokumente, sa používa hodnota s pohyblivou desatinnou čiarkou, ktorá súvisí s frekvenciou výskytu slova v korpuse.

Formálne je váha $w_{ij}$ slova $i$ v dokumente $j$ definovaná ako:
$$
w_{ij} = tf_{ij}\times\log({N\over df_i})
$$
kde
* $tf_{ij}$ je počet výskytov $i$ v $j$, teda hodnota BoW, ktorú sme videli predtým
* $N$ je počet dokumentov v kolekcii
* $df_i$ je počet dokumentov obsahujúcich slovo $i$ v celej kolekcii

Hodnota TF-IDF $w_{ij}$ rastie úmerne počtu výskytov slova v dokumente a je upravená podľa počtu dokumentov v korpuse, ktoré obsahujú dané slovo, čo pomáha zohľadniť fakt, že niektoré slová sa vyskytujú častejšie než iné. Napríklad, ak sa slovo vyskytuje *v každom* dokumente v kolekcii, $df_i=N$, a $w_{ij}=0$, a tieto výrazy by boli úplne ignorované.

TF-IDF vektorizáciu textu môžete jednoducho vytvoriť pomocou Scikit Learn:


In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[0.43381609, 0.        , 0.43381609, 0.        , 0.65985664,
        0.43381609, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ]])

## Záver

Aj keď TF-IDF reprezentácie poskytujú váhovanie frekvencie rôznym slovám, nedokážu zachytiť význam alebo poradie. Ako povedal známy lingvista J. R. Firth v roku 1935: „Úplný význam slova je vždy kontextový a žiadna štúdia významu mimo kontextu nemôže byť braná vážne.“ Neskôr v kurze sa naučíme, ako zachytiť kontextové informácie z textu pomocou jazykového modelovania.



---

**Upozornenie**:  
Tento dokument bol preložený pomocou služby na automatický preklad [Co-op Translator](https://github.com/Azure/co-op-translator). Aj keď sa snažíme o presnosť, upozorňujeme, že automatické preklady môžu obsahovať chyby alebo nepresnosti. Pôvodný dokument v jeho pôvodnom jazyku by mal byť považovaný za autoritatívny zdroj. Pre dôležité informácie sa odporúča profesionálny ľudský preklad. Nezodpovedáme za akékoľvek nedorozumenia alebo nesprávne interpretácie vyplývajúce z použitia tohto prekladu.
